# Prueba de funciones

In [36]:
import pandas as pd
import numpy as np
import time
import sys

In [2]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [3]:
#### Asisting configuration
sys.path.append("..")

In [4]:
#from src.utils.utils import convierte_a_minusculas
from src.pipeline.taks_1_preprocesamiento import preprocesamiento_limpieza

In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [5]:
df = pd.read_csv("../data/ds_challenge_apr2021.csv")

In [6]:
#

In [7]:
df = preprocesamiento_limpieza(df)

In [8]:
#

In [9]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [18]:
def transformacion(data):
     
    standard_scaler = StandardScaler()
    
    transformers = [('one_hot', 
                     OneHotEncoder(sparse = False, 
                                   categories = "auto"),
                     ['tipo_tc', 'is_prime', 'genero', 
                      'establecimiento', 'ciudad']),
                    ('standar_scaler', 
                     standard_scaler, 
                     ['monto', 'dcto','cashback',
                      'linea_tc','interes_tc'
                ])]
    
    col_trans = ColumnTransformer(transformers, remainder="drop",
                                  n_jobs=-1, verbose=True)
    
    df_regressors = col_trans.fit(data)
    df_transformed = df_regressors.transform(data)
    
    A = [sorted(data.tipo_tc.unique()),
         sorted(data.is_prime.unique()),
         sorted(data.genero.unique()),
         sorted(data.establecimiento.unique()),
         sorted(data.ciudad.unique())]
    
    colnames = sum(A, [])
    
    colnames.append('monto')
    colnames.append('dcto')
    colnames.append('cashback')
    colnames.append('linea_tc')
    colnames.append('interes_tc')
    
    final_df = pd.DataFrame(df_transformed, columns = colnames)
    
    final_df['fraude'] = data['fraude']
    final_df['fecha'] = data['fecha']
    final_df['hora'] = data['hora']
    
    
    
    return final_df    

In [19]:
final_df = transformacion(df)

In [20]:
final_df

,fisica,virtual,no_prime,si_prime,f,m,no_def,NA_establecimiento,abarrotes,farmacia,...,monterrey,toluca,monto,dcto,cashback,linea_tc,interes_tc,fraude,fecha,hora
0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.377721,1.263136,-0.176315,0.389427,0.289465,0,2020-01-21,20
1,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,-1.418404,-0.508714,-1.006380,0.389427,0.289465,0,2020-01-15,7
2,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.005752,-0.508714,2.139704,1.440304,0.811001,0,2020-01-17,11
3,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.926139,-0.508714,0.316006,1.440304,0.811001,0,2020-01-05,11
4,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,-0.846509,-0.508714,-0.834270,1.440304,0.811001,0,2020-01-30,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26970,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.854196,-0.508714,0.269322,-1.026972,0.811001,0,2020-01-08,13
26971,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,1.530346,-0.508714,0.708078,-1.026972,0.811001,0,2020-01-14,15
26972,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,-0.856813,-0.508714,-0.277543,-1.026972,0.811001,0,2020-01-16,11
26973,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,-1.641479,-0.438280,-1.355553,-1.026972,0.811001,0,2020-01-13,17


In [21]:
def horas_dias_transformacion(data):
    
    data['dia'] = pd.DatetimeIndex(data['fecha']).day
    
    dias = 30 
    
    data['sin_dia'] = np.sin(2*np.pi*data['dia']/dias)
    data['cos_dia'] = np.cos(2*np.pi*data['dia']/dias)
    
    data = data.drop('fecha',1)
    data = data.drop('dia',1)
    
    horas = 24 
    
    data['sin_hora'] = np.sin(2*np.pi*data['hora']/horas)
    data['cos_hora'] = np.cos(2*np.pi*data['hora']/horas)
    
    data = data.drop('hora',1)
    
    return data

In [22]:
final_df = horas_dias_transformacion(final_df)

In [46]:
final_df

,fisica,virtual,no_prime,si_prime,f,m,no_def,NA_establecimiento,abarrotes,farmacia,...,monto,dcto,cashback,linea_tc,interes_tc,fraude,sin_dia,cos_dia,sin_hora,cos_hora
0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.377721,1.263136,-0.176315,0.389427,0.289465,0,-9.510565e-01,-0.309017,-0.866025,0.500000
1,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,-1.418404,-0.508714,-1.006380,0.389427,0.289465,0,5.665539e-16,-1.000000,0.965926,-0.258819
2,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.005752,-0.508714,2.139704,1.440304,0.811001,0,-4.067366e-01,-0.913545,0.258819,-0.965926
3,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.926139,-0.508714,0.316006,1.440304,0.811001,0,8.660254e-01,0.500000,0.258819,-0.965926
4,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,-0.846509,-0.508714,-0.834270,1.440304,0.811001,0,-1.133108e-15,1.000000,0.500000,-0.866025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26970,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.854196,-0.508714,0.269322,-1.026972,0.811001,0,9.945219e-01,-0.104528,-0.258819,-0.965926
26971,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.530346,-0.508714,0.708078,-1.026972,0.811001,0,2.079117e-01,-0.978148,-0.707107,-0.707107
26972,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,-0.856813,-0.508714,-0.277543,-1.026972,0.811001,0,-2.079117e-01,-0.978148,0.258819,-0.965926
26973,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,-1.641479,-0.438280,-1.355553,-1.026972,0.811001,0,4.067366e-01,-0.913545,-0.965926,-0.258819


In [45]:
final_df.drop('fraude',1).values.shape

(26975, 27)

In [39]:
def feature(data):
    
    y = data['fraude'].values
    X = data.drop('fraude',1).values
    print(X.shape,y.shape)
    
    
    # ocuparemos un RF
    classifier = RandomForestClassifier(oob_score=True, random_state=1234)

    # separando en train, test
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    # definicion de los hiperparametros que queremos probar
    hyper_param_grid = {'n_estimators': [100,200,300], 
                        'max_depth': [1, 2, 3],
                        'min_samples_split': [2, 3, 5, 7]}

    # ocupemos grid search!
    gs = GridSearchCV(classifier, 
                      hyper_param_grid, 
                      scoring = 'precision', #
                      cv = 5, 
                      n_jobs = -1)
    start_time = time.time()
    gs.fit(X_train, y_train)
    
    print("Tiempo de ejecución: ", time.time() - start_time)
    
    colnames = data.columns
    importancias = pd.DataFrame(colnames,
                                gs.best_estimator_.feature_importances_)
    importancias['predictores'] = importancias.loc[:,0]
    importancias.reset_index().iloc[:, [2,0]].sort_values(
        by = 'index', ascending=False)
    

    
    return importancias

In [40]:
importancias=feature(final_df)

(26975, 27) (26975,)
Tiempo de ejecución:  171.13021087646484


ValueError: Shape of passed values is (28, 1), indices imply (27, 1)